In [1]:
%%init_spark
# Configure Spark to use a local master
launcher.master = "local"

In [2]:
import org.apache.spark.sql.functions.{window, column, desc, col,count}
import org.apache.spark.sql.expressions.Window

Intitializing Scala interpreter ...

Spark Web UI available at http://9a9c2e1d7612:4041
SparkContext available as 'sc' (version = 3.0.0, master = local, app id = local-1597110705188)
SparkSession available as 'spark'


import org.apache.spark.sql.functions.{window, column, desc, col, count}
import org.apache.spark.sql.expressions.Window


In [3]:
val df = spark.read.format("csv").option("header", "true").load("data/all_data.csv")

df: org.apache.spark.sql.DataFrame = [producto: string, presentacion: string ... 13 more fields]


In [4]:
df.printSchema()

root
 |-- producto: string (nullable = true)
 |-- presentacion: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- catalogo: string (nullable = true)
 |-- precio: string (nullable = true)
 |-- fechaRegistro: string (nullable = true)
 |-- cadenaComercial: string (nullable = true)
 |-- giro: string (nullable = true)
 |-- nombreComercial: string (nullable = true)
 |-- direccion: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- latitud: string (nullable = true)
 |-- longitud: string (nullable = true)



### 1. ¿Cuántos registros hay?

In [5]:
val totalRegistros = df.count()

totalRegistros: Long = 62530715


In [6]:
totalRegistros

res1: Long = 62530715


### 2. ¿Cuántas categorías?

In [7]:
val diferentesCategorias = df.select("categoria")
                                   .filter("categoria is not NULL")
                                   .distinct().count()

diferentesCategorias: Long = 41


### 3. ¿Cuántas cadenas comerciales están siendo monitoreadas (y, por lo tanto, reportadas en esa base de datos)?

In [8]:
val totalCadenasComerciales = df.select("cadenaComercial")
                                      .filter("cadenaComercial is not NULL")
                                      .distinct().count()

totalCadenasComerciales: Long = 705


### 4. ¿Cuáles son los productos más monitoreados en cada estado de la república?

In [9]:
val estadosProductos = df.where((col("estado").isNotNull) && (col("producto").isNotNull))
                                .groupBy("estado","producto")
                                .agg(count("producto").alias("total"))
                                .select("estado","producto","total")

estadosProductos: org.apache.spark.sql.DataFrame = [estado: string, producto: string ... 1 more field]


In [10]:
estadosProductos.show(5)

+----------------+--------------------+-----+
|          estado|            producto|total|
+----------------+--------------------+-----+
|          MÉXICO|TINTE PARA EL CAB...|44007|
|          MÉXICO|         TELEVISORES|29702|
|          MÉXICO|              ACELGA| 7691|
|          MÉXICO|       QUESO. COTIJA| 4414|
|DISTRITO FEDERAL|              AZUCAR|18078|
+----------------+--------------------+-----+
only showing top 5 rows



In [11]:
val w = Window.partitionBy("estado").orderBy(col("total").desc)

w: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@8d7ab9


In [12]:
val productosMasMonitoreados = estadosProductos.withColumn("rn", row_number().over(w))
                           .filter("rn == 1")
                           .select("estado", "producto","total")
                           .orderBy(col("total").desc)

productosMasMonitoreados: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [estado: string, producto: string ... 1 more field]


In [13]:
productosMasMonitoreados.show()

+--------------------+--------------------+------+
|              estado|            producto| total|
+--------------------+--------------------+------+
|    DISTRITO FEDERAL|            REFRESCO|287463|
|              MÉXICO|            REFRESCO|194939|
|             JALISCO|            REFRESCO| 81718|
|          NUEVO LEÓN|   DETERGENTE P/ROPA| 50307|
|          GUANAJUATO|            REFRESCO| 49441|
|            TLAXCALA|            REFRESCO| 43904|
| MICHOACÁN DE OCAMPO|   DETERGENTE P/ROPA| 40144|
|     BAJA CALIFORNIA|            REFRESCO| 37243|
|             YUCATÁN|LECHE ULTRAPASTEU...| 35991|
|        QUINTANA ROO|                 FUD| 34846|
|              SONORA|            REFRESCO| 34731|
|             SINALOA|            REFRESCO| 33115|
|              PUEBLA|            REFRESCO| 31597|
|             TABASCO|            REFRESCO| 28754|
|COAHUILA DE ZARAGOZA|                 FUD| 28613|
|           QUERÉTARO|LECHE ULTRAPASTEU...| 27868|
| BAJA CALIFORNIA SUR|         

### 5. ¿Cuál es la cadena comercial con mayor variedad de productos monitoreados?

In [14]:
val cadenasComercialesPorProducto= df.filter("producto is not NULL")
                                     .groupBy("cadenaComercial") 
                                     .agg(countDistinct("producto").alias("totalProductos"))
                                     .orderBy(desc("totalProductos")).limit(1)

cadenasComercialesPorProducto: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cadenaComercial: string, totalProductos: bigint]


In [15]:
cadenasComercialesPorProducto.show(1)

+---------------+--------------+
|cadenaComercial|totalProductos|
+---------------+--------------+
|        SORIANA|          1059|
+---------------+--------------+



In [ ]:
spark.stop()